In [2]:
import pandas as pd
df = pd.read_pickle('../data/df_final_v2.pkl')
print(df.shape)
df.head(2)

(50357, 13)


,index,id,title,text,dateCreate,product,sub_product,source,text_clean,review,predicted_classes_new,predicted_classes_refined,final_classes
0,0,992651,не выполняют условия акции,в апреле 2025 года я рекомендовала дебетовую к...,2025-08-29T23:30:38.746003Z,debitCards,Умная карта (Премиум),sravni,в апреле года я рекомендовала дебетовую карту ...,не выполняют условия акции в апреле года я ре...,"[1, 3, 14, 15]","[1, 3, 14, 15]","[debitCards, cashbackPromo, notifications, ref..."
1,1,998360,жалоба на услугу газпром бонус премиум,купил услугу газпром бонус премиум за 2 цель б...,2025-09-15T09:38:13.34818Z,debitCards,Отсутствует,sravni,купил услугу газпром бонус премиум за цель был...,жалоба на услугу газпром бонус премиум купил ...,[],[],[debitCards]


In [7]:
'[сбер, сбербанк, sber, sberbank, сберегательный банк, сберик, сбербанк]'

'[сбер, сбербанк, sber, sberbank, сберегательный банк, сберик, сбербанк]'

In [55]:
banks = [
    ['газпромбанк', 'gazprombank', 'гпб'],
    ['сбербанк', 'сбер', 'sber', 'sberbank', 'сберегательный банк', 'сберик'],
    ['втб', 'vtb', 'втб банк', 'vtb bank'],
    ['альфа', 'альфа банк', 'альфа-банк', 'alpha', 'alpha bank', 'alpha-bank', 'альфабанк'],
    ['тинькофф', 'tinkoff', 'тинькофф банк', 'tinkoff bank', 'тиньк', 'тинька'],
    ['райффайзен', 'райффайзенбанк', 'райфайзен', 'райфайзенбанк', 'raiffeisen', 'raiffeisenbank'],
    ['совкомбанк', 'sovcombank'],
    ['промсвязьбанк', 'псб', 'psb', 'promsvyazbank'],
    ['россельхозбанк', 'рсхб', 'rshb', 'rosselkhozbank'],
    ['ак барс', 'акбарс', 'ак барс банк', 'ak bars'],
    ['ренессанс кредит', 'ренессанс', 'renaissance credit'],
    ['совгазбанк', 'sovgazbank'],
    ['абсолют', 'абсолют банк', 'absolut bank'],
    ['мтс', 'мтс банк', 'mts bank'],
    ['точка', 'банк точка', 'tochka', 'rocketbank'],
    ['почта банк', 'почтабанк', 'pochta bank']
]
competitor_banks = [b for b in banks if b[0] != 'газпромбанк']

In [64]:
from itertools import chain
competitor_bank_tokens = [(t[0], set(t)) for t in competitor_banks]

def get_competitor_banks(text: str):
    tokens = set(text.split())
    res = []
    for bank, bank_tokens in competitor_bank_tokens:
        if any(btoken in tokens for btoken in bank_tokens):
            res.append(bank)
    return ','.join(sorted(res))

In [65]:
df['competitor_banks'] = df['text_clean'].apply(get_competitor_banks)
df['has_competitor_banks'] = df['competitor_banks'].apply(lambda x: len(x) > 0)

In [66]:
df.groupby('has_competitor_banks').agg({'index': 'count'})

,index
has_competitor_banks,
False,47823
True,2534


In [68]:
df[df['has_competitor_banks'] == True].sample(2)[['competitor_banks', 'text']].values

array([['мтс',
        'решил стать клиентом газпромбанка в условиях быстро меняющихся ставок по вкладам удобно иметь широкий выбор условий от разных банков заказал на сайте оформление дебетовой карты через несколько дней приехал курьер оформил все документы и выдал карту далее необходимо установить мобильный банк после запуска приложение просит ввести номер телефона и код из смс вот только смс не приходят при курьере при оформлении все приходило по телефону горячей линии ожидание 30 мин зато на сайте есть форма обратной связи написал обращение при отправке просит ввести номер телефона и код из смс я уже думал тупик но код пришел через день служба поддержки ответила что нужно придти в офис и актуализировать какие то данные бред ну хорошо хоть не сказали что это у меня проблемы с приемом смс специально взял распечатку из мтс и никаких входящих смс от газпромбанка после визита курьера нет ну кроме одной когда обращение писал не вижу смысла идти к ним в офис тем более что в отзывах люди п

In [67]:
df[['id', 'competitor_banks', 'has_competitor_banks']].to_csv('../data/competitors_data.csv', index=False)